In [788]:
import pandas as pd
import altair as alt
import re
import glob
import os

import gspread
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
from gspread_dataframe import set_with_dataframe

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl.worksheet.header_footer")

# Setup for google upload (optional)

In [790]:
#url to google spreadsheet that feeds the data to the charts
SPREADSHEET_URL = "YOUR URL HERE"

In [791]:
# Google Sheets API setup
def authenticate_gspread():
    # Define the scope
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    
    # Provide the path to your Google Service Account JSON key file
    credentials = ServiceAccountCredentials.from_json_keyfile_name('YOUR KEYFILE HERE', scope)
    
    # Authenticate and initialize the client
    client = gspread.authorize(credentials)
    
    return client

In [792]:
# Function to upload the dataframe to a specific tab in the spreadsheet
def upload_to_google_sheets(spreadsheet_url, sheet_name, dataframe):
    # Authenticate the client
    client = authenticate_gspread()
    
    # Open the spreadsheet by URL
    spreadsheet = client.open_by_url(spreadsheet_url)
    
    # Select the specific worksheet (tab) by name
    worksheet = spreadsheet.worksheet(sheet_name)
    
    # Clear the existing content in the worksheet
    worksheet.clear()
    
    # Write the dataframe to the worksheet
    set_with_dataframe(worksheet, dataframe, include_index = False, include_column_header=True, resize=False)

# Reading in data

In [793]:
# create list of filenames
folder_path = '../data/data-unified/EC-HomeAffairs_VisaConsulates/'
files_EC_consulates = glob.glob(os.path.join(folder_path, '*.xlsx*'))
files_EC_consulates

['../data/data-unified/EC-HomeAffairs_VisaConsulates/EC-HA_ConsulatesSchengenVisa-2020.xlsx',
 '../data/data-unified/EC-HomeAffairs_VisaConsulates/EC-HA_ConsulatesSchengenVisa-2016.xlsx',
 '../data/data-unified/EC-HomeAffairs_VisaConsulates/EC-HA_ConsulatesSchengenVisa-2017.xlsx',
 '../data/data-unified/EC-HomeAffairs_VisaConsulates/EC-HA_ConsulatesSchengenVisa-2021.xlsx',
 '../data/data-unified/EC-HomeAffairs_VisaConsulates/EC-HA_ConsulatesSchengenVisa-2024.xlsx',
 '../data/data-unified/EC-HomeAffairs_VisaConsulates/EC-HA_ConsulatesSchengenVisa-2018.xlsx',
 '../data/data-unified/EC-HomeAffairs_VisaConsulates/EC-HA_ConsulatesSchengenVisa-2022.xlsx',
 '../data/data-unified/EC-HomeAffairs_VisaConsulates/EC-HA_ConsulatesSchengenVisa-2014.xlsx',
 '../data/data-unified/EC-HomeAffairs_VisaConsulates/EC-HA_ConsulatesSchengenVisa-2015.xlsx',
 '../data/data-unified/EC-HomeAffairs_VisaConsulates/EC-HA_ConsulatesSchengenVisa-2023.xlsx',
 '../data/data-unified/EC-HomeAffairs_VisaConsulates/EC-HA_C

In [794]:
EC_CONSULATE_DATA = pd.DataFrame() #creating a dataframe to feed all data into

In [795]:
#columns of interest
DATASET_COLUMNS_SELECTED = ['Schengen State', 'Country where consulate is located', 'Consulate','Uniform visas applied for','Total uniform visas issued (including MEV) \n','Multiple entry uniform visas (MEVs) issued','Share of MEVs on total number of uniform visas issued','Uniform visas not issued','Not issued rate for uniform visas']

In [796]:
#function to read in yearly dataset
def read_excel_df(filename):
    df_year = pd.read_excel(filename, sheet_name='Data for consulates', usecols=DATASET_COLUMNS_SELECTED) #read in one file
    year_str= re.sub(r"(.*)(\d\d\d\d)(.*)",r"\2",filename) #grab year from filename
    df_year['year']=year_str #save year as a separate column to df
    return df_year

In [797]:
# deleted info

#2014
#1) For Germany: only MEVs valid for more than 1 year are included
#2) The figure for the Finnish Embassy in Pristina includes 293 multiple-entry LTVs issued 

#2015
#1) for visas issued by Germany: only MEVs valid for more than 1 year are included

In [798]:
for file in files_EC_consulates:
    print(file)
    df_yearly = read_excel_df(file)
    EC_CONSULATE_DATA=pd.concat([EC_CONSULATE_DATA,df_yearly])

../data/data-unified/EC-HomeAffairs_VisaConsulates/EC-HA_ConsulatesSchengenVisa-2020.xlsx
../data/data-unified/EC-HomeAffairs_VisaConsulates/EC-HA_ConsulatesSchengenVisa-2016.xlsx
../data/data-unified/EC-HomeAffairs_VisaConsulates/EC-HA_ConsulatesSchengenVisa-2017.xlsx
../data/data-unified/EC-HomeAffairs_VisaConsulates/EC-HA_ConsulatesSchengenVisa-2021.xlsx
../data/data-unified/EC-HomeAffairs_VisaConsulates/EC-HA_ConsulatesSchengenVisa-2024.xlsx
../data/data-unified/EC-HomeAffairs_VisaConsulates/EC-HA_ConsulatesSchengenVisa-2018.xlsx
../data/data-unified/EC-HomeAffairs_VisaConsulates/EC-HA_ConsulatesSchengenVisa-2022.xlsx
../data/data-unified/EC-HomeAffairs_VisaConsulates/EC-HA_ConsulatesSchengenVisa-2014.xlsx
../data/data-unified/EC-HomeAffairs_VisaConsulates/EC-HA_ConsulatesSchengenVisa-2015.xlsx
../data/data-unified/EC-HomeAffairs_VisaConsulates/EC-HA_ConsulatesSchengenVisa-2023.xlsx
../data/data-unified/EC-HomeAffairs_VisaConsulates/EC-HA_ConsulatesSchengenVisa-2019.xlsx


In [799]:
EC_CONSULATE_DATA.head()

,Schengen State,Country where consulate is located,Consulate,Uniform visas applied for,Total uniform visas issued (including MEV) \n,Multiple entry uniform visas (MEVs) issued,Share of MEVs on total number of uniform visas issued,Uniform visas not issued,Not issued rate for uniform visas,year
0,Austria,ALBANIA,TIRANA,14.0,9.0,9.0,1.000000,0.0,0.000000,2020
1,Austria,ALGERIA,ALGIERS,457.0,358.0,253.0,0.706704,99.0,0.216630,2020
2,Austria,ARGENTINA,BUENOS AIRES,4.0,4.0,3.0,0.750000,NaN,0.000000,2020
3,Austria,AUSTRALIA,CANBERRA,372.0,371.0,371.0,1.000000,1.0,0.002688,2020
4,Austria,AZERBAIJAN,BAKU,343.0,323.0,278.0,0.860681,14.0,0.040816,2020


In [800]:
#renaming columns
EC_CONSULATE_DATA.columns=(['SchengenState', 'ConsulateLocation_country', 'Consulate',
       'Uniform_visa_applications',
       'Uniform_visas_issued_total',
       'MEV_issued',
       'Share_MEV_in_total_issued',
       'Visas_not_issued', 'Not_issued_rate',
       'year'])

In [801]:
EC_CONSULATE_DATA.head()

,SchengenState,ConsulateLocation_country,Consulate,Uniform_visa_applications,Uniform_visas_issued_total,MEV_issued,Share_MEV_in_total_issued,Visas_not_issued,Not_issued_rate,year
0,Austria,ALBANIA,TIRANA,14.0,9.0,9.0,1.000000,0.0,0.000000,2020
1,Austria,ALGERIA,ALGIERS,457.0,358.0,253.0,0.706704,99.0,0.216630,2020
2,Austria,ARGENTINA,BUENOS AIRES,4.0,4.0,3.0,0.750000,NaN,0.000000,2020
3,Austria,AUSTRALIA,CANBERRA,372.0,371.0,371.0,1.000000,1.0,0.002688,2020
4,Austria,AZERBAIJAN,BAKU,343.0,323.0,278.0,0.860681,14.0,0.040816,2020


In [857]:
#after inspection, filtering out rows that do not contain information related to consulates/countries
EC_CONSULATE_DATA = EC_CONSULATE_DATA[EC_CONSULATE_DATA['ConsulateLocation_country'].notnull()]

In [858]:
#adding information on regions
REGIONS = pd.read_csv('../data/country-and-continent-codes-list.csv',delimiter=';') #reading in regions data
REGIONS = REGIONS[['continent','region','country','ISO-alpha3 code']] #selecting needed columns
REGIONS.columns=(['continent','region','country','ISO3code']) #renaming columns
REGIONS['ConsulateLocation_country'] = [value.upper() for value in REGIONS['country']] # convert country names to upper case to match visa data
REGIONS.head()

,continent,region,country,ISO3code,ConsulateLocation_country
0,Asia,Southern Asia,Afghanistan,AFG,AFGHANISTAN
1,Europe,Southern Europe,Albania,ALB,ALBANIA
2,Americas,Northern America,Antarctica,ATA,ANTARCTICA
3,Africa,Northern Africa,Algeria,DZA,ALGERIA
4,Oceania,Polynesia,American Samoa,ASM,AMERICAN SAMOA


In [859]:
EC_CONSULATE_DATA_regionalized = EC_CONSULATE_DATA.merge(REGIONS, on='ConsulateLocation_country')

In [860]:
print(len(EC_CONSULATE_DATA))
print(len(EC_CONSULATE_DATA_regionalized))

20338
20336


In [861]:
len(EC_CONSULATE_DATA)-len(EC_CONSULATE_DATA_regionalized)

2

In [862]:
before_merge_country_list = EC_CONSULATE_DATA.ConsulateLocation_country.unique()

In [863]:
after_merge_country_list = EC_CONSULATE_DATA_regionalized.ConsulateLocation_country.unique()

In [864]:
difference = list(set(before_merge_country_list) - set(after_merge_country_list))
difference

['HOLY SEE (VATICAN CITY STATE)']

In [865]:
len(EC_CONSULATE_DATA[EC_CONSULATE_DATA['ConsulateLocation_country']=='HOLY SEE (VATICAN CITY STATE)'])

2

In [866]:
# aggregate by issuing Schengen state and year
VISAS_YEARLY = EC_CONSULATE_DATA.groupby(by=['SchengenState','year']).agg({
    'Uniform_visa_applications': 'sum',
    'Uniform_visas_issued_total': 'sum',
    'MEV_issued': 'sum',
    'Visas_not_issued': 'sum',
}).reset_index()

In [867]:
VISAS_YEARLY.head()

,SchengenState,year,Uniform_visa_applications,Uniform_visas_issued_total,MEV_issued,Visas_not_issued
0,Austria,2014,266356.0,258247.0,172804.0,7204.0
1,Austria,2015,259167.0,247800.0,145611.0,8564.0
2,Austria,2016,268388.0,257401.0,146035.0,8173.0
3,Austria,2017,304556.0,284904.0,153159.0,14233.0
4,Austria,2018,306133.0,280847.0,154902.0,19098.0


In [868]:
SCHENGENAVERAGE = VISAS_YEARLY.groupby('year').agg({
    'Uniform_visa_applications': 'sum',
    'Uniform_visas_issued_total': 'sum',
    'MEV_issued': 'sum',
    'Visas_not_issued': 'sum',
}).reset_index()

SCHENGENAVERAGE['SchengenState']='Schengen Average'
SCHENGENAVERAGE.head()

,year,Uniform_visa_applications,Uniform_visas_issued_total,MEV_issued,Visas_not_issued,SchengenState
0,2014,16725908.0,15684796.0,7530725.0,851010.0,Schengen Average
1,2015,15446676.0,14308392.0,6942449.0,959650.0,Schengen Average
2,2016,15192502.0,13937715.0,8169007.0,1043549.0,Schengen Average
3,2017,16155613.0,14653724.0,8677376.0,1317451.0,Schengen Average
4,2018,16016599.0,14265282.0,8325850.0,1539362.0,Schengen Average


In [869]:
VISAS_YEARLY = pd.concat([VISAS_YEARLY, SCHENGENAVERAGE])

In [870]:
VISAS_YEARLY['Not_issued_rate_yearly']= VISAS_YEARLY['Visas_not_issued']/VISAS_YEARLY['Uniform_visa_applications']

In [871]:
# does any Schengen state drastically change with regards to their rejection rates?

alt.Chart(VISAS_YEARLY).mark_line().encode(
    x='year',
    y='Not_issued_rate_yearly',
    color='SchengenState',
    tooltip=['SchengenState','Not_issued_rate_yearly'],
).properties(width=500)

alt.Chart(...)

In [872]:
#status quon in 2024
VISAS_YEARLY[VISAS_YEARLY['year']=='2024'].sort_values(by='Not_issued_rate_yearly', ascending = False)

,SchengenState,year,Uniform_visa_applications,Uniform_visas_issued_total,MEV_issued,Visas_not_issued,Not_issued_rate_yearly
178,Malta,2024,45578.0,26837.0,13110.0,16905.0,0.370903
57,Estonia,2024,12125.0,8317.0,4030.0,3291.0,0.271423
245,Slovenia,2024,18171.0,13562.0,9232.0,4417.0,0.243080
21,Belgium,2024,255564.0,188766.0,124487.0,61724.0,0.241521
267,Sweden,2024,188623.0,140422.0,30440.0,44576.0,0.236323
46,Denmark,2024,132158.0,99745.0,41993.0,31013.0,0.234666
24,Croatia,2024,42165.0,33441.0,24223.0,8003.0,0.189802
211,Poland,2024,111538.0,91905.0,41791.0,19277.0,0.172829
35,Czech Republic,2024,150629.0,126628.0,43903.0,23735.0,0.157573
79,France,2024,3072728.0,2549735.0,917660.0,481139.0,0.156584


# by continent

In [873]:
#aggregate by continent and year
YEARLY_CONTINENT = EC_CONSULATE_DATA_regionalized.groupby(by=['continent','year']).agg({
    'Uniform_visa_applications': 'sum',
    'Uniform_visas_issued_total': 'sum',
    'MEV_issued': 'sum',
    'Visas_not_issued': 'sum',
}).reset_index()

YEARLY_CONTINENT.head()

,continent,year,Uniform_visa_applications,Uniform_visas_issued_total,MEV_issued,Visas_not_issued
0,Africa,2014,2221958.0,1769574.0,697367.0,411064.0
1,Africa,2015,2468526.0,1960370.0,811388.0,467512.0
2,Africa,2016,2567870.0,1978660.0,876375.0,516911.0
3,Africa,2017,2761920.0,2071382.0,874322.0,655607.0
4,Africa,2018,2823866.0,2017674.0,850544.0,757858.0


In [874]:
#calculate rejection rate per year /continent
YEARLY_CONTINENT['Not_issued_rate_yearly']= YEARLY_CONTINENT['Visas_not_issued']/YEARLY_CONTINENT['Uniform_visa_applications']

In [875]:
YEARLY_CONTINENT.head()

,continent,year,Uniform_visa_applications,Uniform_visas_issued_total,MEV_issued,Visas_not_issued,Not_issued_rate_yearly
0,Africa,2014,2221958.0,1769574.0,697367.0,411064.0,0.185001
1,Africa,2015,2468526.0,1960370.0,811388.0,467512.0,0.189389
2,Africa,2016,2567870.0,1978660.0,876375.0,516911.0,0.201300
3,Africa,2017,2761920.0,2071382.0,874322.0,655607.0,0.237374
4,Africa,2018,2823866.0,2017674.0,850544.0,757858.0,0.268376


In [876]:
#tp250206
#todo SSA als eigene Linie zusätzlich zu "Africa"
alt.Chart(YEARLY_CONTINENT).mark_line().encode(
    x='year',
    y='Not_issued_rate_yearly',
    color='continent',
    tooltip=['continent','Not_issued_rate_yearly'],
).properties(width=500)

alt.Chart(...)

In [877]:
#adding SSA as an individual line
YEARLY_SSA_rejection=YEARLY_REGION[YEARLY_REGION['region']=='Sub-Saharan Africa'][['region','year','Not_issued_rate_yearly']]
YEARLY_SSA_rejection.columns=(['continent','year','Not_issued_rate_yearly'])
YEARLY_SSA_rejection.head()

,continent,year,Not_issued_rate_yearly
153,Sub-Saharan Africa,2014,0.194259
154,Sub-Saharan Africa,2015,0.186069
155,Sub-Saharan Africa,2016,0.208598
156,Sub-Saharan Africa,2017,0.218359
157,Sub-Saharan Africa,2018,0.227779


In [878]:
YEARLY_CONTINENT_rejection = YEARLY_CONTINENT[['continent','year','Not_issued_rate_yearly']]
YEARLY_CONTINENT_rejection.head()

,continent,year,Not_issued_rate_yearly
0,Africa,2014,0.185001
1,Africa,2015,0.189389
2,Africa,2016,0.201300
3,Africa,2017,0.237374
4,Africa,2018,0.268376


In [879]:
YEARLY_CONTINENT_SSA_rejection=pd.concat([YEARLY_CONTINENT_rejection,YEARLY_SSA_rejection])

In [880]:
#reporting-relevant
alt.Chart(YEARLY_CONTINENT_SSA_rejection).mark_line().encode(
    x='year',
    y='Not_issued_rate_yearly',
    color='continent',
    tooltip=['continent','Not_issued_rate_yearly'],
).properties(width=500)

alt.Chart(...)

In [881]:
#reconfiguring dataset for datawrapper - each region to be a column

YEARLY_CONTINENT_SSA_rejection_pivoted = YEARLY_CONTINENT_SSA_rejection.pivot(index='year',columns='continent', values='Not_issued_rate_yearly')
YEARLY_CONTINENT_SSA_rejection_pivoted = YEARLY_CONTINENT_SSA_rejection_pivoted.reset_index()
YEARLY_CONTINENT_SSA_rejection_pivoted.head()


continent,year,Africa,Americas,Asia,Europe,Oceania,Sub-Saharan Africa
0,2014,0.185001,0.067666,0.052181,0.013664,0.018998,0.194259
1,2015,0.189389,0.062680,0.052153,0.019656,0.014127,0.186069
2,2016,0.201300,0.089635,0.058580,0.019240,0.013713,0.208598
3,2017,0.237374,0.101131,0.071052,0.018627,0.025677,0.218359
4,2018,0.268376,0.111537,0.082109,0.019427,0.023732,0.227779


In [829]:
upload_to_google_sheets(SPREADSHEET_URL, "Rejection-Rates-timeline",YEARLY_CONTINENT_SSA_rejection_pivoted)

In [882]:
alt.Chart(YEARLY_CONTINENT).mark_line().encode(
    x='year',
    y='Uniform_visa_applications',
    color='continent',
    tooltip=['continent','Not_issued_rate_yearly'],
).properties(width=500)

alt.Chart(...)

In [883]:
#tp250602
alt.Chart(YEARLY_CONTINENT[YEARLY_CONTINENT['year']=='2024']).mark_arc(innerRadius=50).encode(
    theta=alt.Theta(field="Uniform_visa_applications", type="quantitative"),
    color=alt.Color(field="continent", type="nominal"),
)

alt.Chart(...)

In [833]:
YEARLY_CONTINENT[YEARLY_CONTINENT['year']=='2024']

,continent,year,Uniform_visa_applications,Uniform_visas_issued_total,MEV_issued,Visas_not_issued,Not_issued_rate_yearly
10,Africa,2024,2648411.0,1853534.0,895704.0,705150.0,0.266254
21,Americas,2024,514367.0,422151.0,179359.0,80190.0,0.155900
32,Asia,2024,7184476.0,6280574.0,3415461.0,831812.0,0.115779
43,Europe,2024,1349169.0,1217706.0,612081.0,88208.0,0.065380
54,Oceania,2024,20300.0,19669.0,9435.0,643.0,0.031675


# by subregion

In [835]:
YEARLY_REGION = EC_CONSULATE_DATA_regionalized.groupby(by=['region','year']).agg({
    'Uniform_visa_applications': 'sum',
    'Uniform_visas_issued_total': 'sum',
    'MEV_issued': 'sum',
    'Visas_not_issued': 'sum',
}).reset_index()

YEARLY_REGION.head()

,region,year,Uniform_visa_applications,Uniform_visas_issued_total,MEV_issued,Visas_not_issued
0,Australia and New Zealand,2014,11165.0,10728.0,4192.0,233.0
1,Australia and New Zealand,2015,14357.0,13882.0,4378.0,242.0
2,Australia and New Zealand,2016,14707.0,14464.0,3742.0,121.0
3,Australia and New Zealand,2017,17831.0,17284.0,4590.0,474.0
4,Australia and New Zealand,2018,19764.0,19190.0,6000.0,472.0


In [836]:
YEARLY_REGION['Not_issued_rate_yearly']= YEARLY_REGION['Visas_not_issued']/YEARLY_REGION['Uniform_visa_applications']

In [837]:
alt.Chart(YEARLY_REGION).mark_line().encode(
    x='year',
    y='Not_issued_rate_yearly',
    color='region',
    tooltip=['region','Not_issued_rate_yearly'],
).properties(width=500)

alt.Chart(...)

In [885]:
#tp250603
YEARLY_REGION[YEARLY_REGION['year']=='2024'].sort_values(by='Not_issued_rate_yearly',ascending = False)

,region,year,Uniform_visa_applications,Uniform_visas_issued_total,MEV_issued,Visas_not_issued,Not_issued_rate_yearly
119,South America,2024,155088.0,108397.0,41129.0,44617.0,0.287688
163,Sub-Saharan Africa,2024,1075448.0,739896.0,355549.0,306800.0,0.285276
86,Northern Africa,2024,1572963.0,1113638.0,540155.0,398350.0,0.253248
65,Latin America and the Caribbean,2024,110634.0,80453.0,19922.0,25080.0,0.226693
141,Southern Asia,2024,1421501.0,1133081.0,676192.0,275782.0,0.194008
174,Western Asia,2024,2782266.0,2365564.0,1647588.0,381337.0,0.137060
32,Central Asia,2024,278048.0,242761.0,91593.0,31484.0,0.113232
152,Southern Europe,2024,33029.0,29266.0,15069.0,2304.0,0.069757
108,Northern Europe,2024,508185.0,467494.0,230750.0,33772.0,0.066456
54,Eastern Europe,2024,783545.0,710096.0,364931.0,51994.0,0.066357


# Subsahara-Africa

In [839]:
#filter data to only include SSA
EC_CONSULATE_DATA_SSA = EC_CONSULATE_DATA_regionalized[EC_CONSULATE_DATA_regionalized['region']=='Sub-Saharan Africa']
EC_CONSULATE_DATA_SSA.head()

,SchengenState,ConsulateLocation_country,Consulate,Uniform_visa_applications,Uniform_visas_issued_total,MEV_issued,Share_MEV_in_total_issued,Visas_not_issued,Not_issued_rate,year,continent,region,country,ISO3code
17,Austria,ETHIOPIA,ADDIS ABEBA,201.0,193.0,20.0,0.103627,5.0,0.024876,2020,Africa,Sub-Saharan Africa,Ethiopia,ETH
28,Austria,KENYA,NAIROBI,202.0,181.0,33.0,0.182320,21.0,0.103960,2020,Africa,Sub-Saharan Africa,Kenya,KEN
34,Austria,NIGERIA,ABUJA,266.0,162.0,38.0,0.234568,82.0,0.308271,2020,Africa,Sub-Saharan Africa,Nigeria,NGA
42,Austria,SENEGAL,DAKAR,125.0,78.0,32.0,0.410256,47.0,0.376000,2020,Africa,Sub-Saharan Africa,Senegal,SEN
46,Austria,SOUTH AFRICA,PRETORIA,1103.0,1010.0,997.0,0.987129,12.0,0.010879,2020,Africa,Sub-Saharan Africa,South Africa,ZAF


In [840]:
# Which Schengen countries in SSA have highest rejection rates in 2024?
SCHENGEN_REJECTION_RATES_2024=EC_CONSULATE_DATA_SSA[EC_CONSULATE_DATA_SSA['year']=='2024'].groupby(by='SchengenState').agg({
    'Uniform_visa_applications': 'sum',
    'Uniform_visas_issued_total': 'sum',
    'MEV_issued': 'sum',
    'Visas_not_issued': 'sum',
}).reset_index()

SCHENGEN_REJECTION_RATES_2024['Not_issued_rate']=SCHENGEN_REJECTION_RATES_2024['Visas_not_issued']/SCHENGEN_REJECTION_RATES_2024['Uniform_visa_applications']
SCHENGEN_REJECTION_RATES_2024.sort_values(by='Not_issued_rate',ascending = False)

,SchengenState,Uniform_visa_applications,Uniform_visas_issued_total,MEV_issued,Visas_not_issued,Not_issued_rate
14,Malta,1118.0,438.0,145.0,653.0,0.584079
5,Denmark,11105.0,4444.0,710.0,6197.0,0.558037
13,Luxembourg,358.0,197.0,57.0,148.0,0.413408
17,Poland,4831.0,2849.0,530.0,1962.0,0.406127
15,Netherlands,65171.0,41088.0,27386.0,21732.0,0.333461
7,France,372481.0,243396.0,78890.0,124114.0,0.333209
1,Belgium,99685.0,65020.0,43329.0,32224.0,0.323258
2,Bulgaria,1474.0,778.0,251.0,465.0,0.315468
22,Sweden,14506.0,9304.0,2085.0,4557.0,0.314146
6,Finland,5701.0,3817.0,1876.0,1752.0,0.307315


In [841]:
#tp250620
bars = alt.Chart(SCHENGEN_REJECTION_RATES_2024.sort_values(by='Not_issued_rate', ascending=False)).mark_bar().encode(
    alt.Y('SchengenState', sort='-x', title=''),
    alt.X('Not_issued_rate')
)

text = alt.Chart(SCHENGEN_REJECTION_RATES_2024.sort_values(by='Not_issued_rate', ascending=False)).mark_text(
    align='left',
    baseline='middle',
    dx=3  # small offset to the right of the bar
).encode(
    alt.Y('SchengenState', sort='-x'),
    alt.X('Not_issued_rate'),
    text=alt.Text('Not_issued_rate:Q', format='.1%')  # adjust format as needed
)

chart = bars + text
chart.properties(title='2024 visa rejection rates* for visas filed in SSA by issuing Schengen state')

alt.LayerChart(...)

In [842]:
#tp250620
visa_applications = alt.Chart(SCHENGEN_REJECTION_RATES_2024).mark_bar().encode(
    alt.Y('SchengenState', sort=alt.EncodingSortField(field='Uniform_visa_applications', order='descending')),
    x='Uniform_visa_applications'
)

visa_rejections = alt.Chart(SCHENGEN_REJECTION_RATES_2024).mark_bar(height=5, color='red').encode(
    alt.Y('SchengenState', sort=alt.EncodingSortField(field='Uniform_visa_applications', order='descending'), title=''),
    x='Visas_not_issued'
).properties(
    title='Absolute number of SSA short stay Schengen visa applications (blue) and rejected visas* (red) by Schengen state'
)

(visa_applications + visa_rejections)

alt.LayerChart(...)

In [843]:
upload_to_google_sheets(SPREADSHEET_URL, 'SSA-Visa-Issuer-2024',SCHENGEN_REJECTION_RATES_2024.sort_values(by='Uniform_visa_applications', ascending = False))

In [844]:
#alltime
SCHENGEN_REJECTION_RATES_alltime=EC_CONSULATE_DATA_SSA.groupby(by='SchengenState').agg({
    'Uniform_visa_applications': 'sum',
    'Uniform_visas_issued_total': 'sum',
    'MEV_issued': 'sum',
    'Visas_not_issued': 'sum',
}).reset_index()

SCHENGEN_REJECTION_RATES_alltime['Not_issued_rate']=SCHENGEN_REJECTION_RATES_alltime['Visas_not_issued']/SCHENGEN_REJECTION_RATES_alltime['Uniform_visa_applications']
SCHENGEN_REJECTION_RATES_alltime.sort_values(by='Not_issued_rate',ascending = False)

,SchengenState,Uniform_visa_applications,Uniform_visas_issued_total,MEV_issued,Visas_not_issued,Not_issued_rate
14,Malta,2403.0,841.0,341.0,1490.0,0.620058
13,Luxembourg,358.0,197.0,57.0,148.0,0.413408
2,Bulgaria,1474.0,778.0,251.0,465.0,0.315468
5,Denmark,89455.0,57478.0,22586.0,27276.0,0.304913
1,Belgium,634151.0,419567.0,192510.0,191163.0,0.301447
17,Poland,47506.0,33374.0,13026.0,13852.0,0.291584
10,Hungary,39152.0,27630.0,12373.0,11245.0,0.287214
7,France,3155378.0,2246876.0,674209.0,849118.0,0.269102
21,Spain,901707.0,614620.0,116012.0,228533.0,0.253445
6,Finland,43577.0,31938.0,14491.0,10607.0,0.243408


In [845]:
#rejection rates by country for all schengen state
SSA_country_yearly = EC_CONSULATE_DATA_SSA.groupby(by=['ISO3code','year']).agg({
    'Uniform_visa_applications': 'sum',
    'Uniform_visas_issued_total': 'sum',
    'MEV_issued': 'sum',
    'Visas_not_issued': 'sum',
}).reset_index()

SSA_country_yearly['Not_issued_rate']=SSA_country_yearly['Visas_not_issued']/SSA_country_yearly['Uniform_visa_applications']
SSA_country_yearly['issuer']='All Schengen countries'
SSA_country_yearly.head()

,ISO3code,year,Uniform_visa_applications,Uniform_visas_issued_total,MEV_issued,Visas_not_issued,Not_issued_rate,issuer
0,AGO,2014,78086.0,63358.0,41331.0,12849.0,0.164549,All Schengen countries
1,AGO,2015,76370.0,63382.0,41396.0,12917.0,0.169137,All Schengen countries
2,AGO,2016,81996.0,67401.0,40124.0,14186.0,0.173008,All Schengen countries
3,AGO,2017,109683.0,88009.0,50847.0,21569.0,0.196649,All Schengen countries
4,AGO,2018,107541.0,84178.0,45121.0,23127.0,0.215053,All Schengen countries


In [846]:
#repeat previous steps for Germany
EC_CONSULATE_DATA_SSA_DE = EC_CONSULATE_DATA_SSA[EC_CONSULATE_DATA_SSA['SchengenState']=='Germany']

SSA_country_DE_yearly = EC_CONSULATE_DATA_SSA_DE.groupby(by=['ISO3code','year']).agg({
    'Uniform_visa_applications': 'sum',
    'Uniform_visas_issued_total': 'sum',
    'MEV_issued': 'sum',
    'Visas_not_issued': 'sum',
}).reset_index()

SSA_country_DE_yearly['Not_issued_rate']=SSA_country_DE_yearly['Visas_not_issued']/SSA_country_DE_yearly['Uniform_visa_applications']
SSA_country_DE_yearly['issuer']='Germany'
SSA_country_DE_yearly.head()

,ISO3code,year,Uniform_visa_applications,Uniform_visas_issued_total,MEV_issued,Visas_not_issued,Not_issued_rate,issuer
0,AGO,2014,2706.0,1916.0,98.0,790.0,0.291944,Germany
1,AGO,2015,2027.0,1298.0,40.0,729.0,0.359645,Germany
2,AGO,2016,1478.0,953.0,249.0,523.0,0.353857,Germany
3,AGO,2017,1850.0,1090.0,238.0,758.0,0.409730,Germany
4,AGO,2018,2113.0,1275.0,346.0,837.0,0.396119,Germany


In [847]:
SSA_country_yearly_issuer = pd.concat([SSA_country_DE_yearly,SSA_country_yearly])

In [848]:
#reporting-relevant
alt.Chart(SSA_country_yearly_issuer).mark_line().encode(
    alt.X('year',title=''),
    y='Not_issued_rate',
    color='issuer:N',
    facet = alt.Facet('ISO3code', columns=7, title='')
).properties(width=100, height = 100, title='Share of rejected visa applications* by location of consulate and by Germany in particular vs all Schengen states')

alt.Chart(...)

In [ ]:
#reconfiguring data for datawrapper

In [850]:
SSA_country_Schengen_not_issued = SSA_country_yearly[['ISO3code','year','Not_issued_rate']]
SSA_country_Schengen_not_issued.columns=(['ISO3code','year','Not_issued_rate_Schengen'])
SSA_country_Schengen_not_issued.head()

,ISO3code,year,Not_issued_rate_Schengen
0,AGO,2014,0.164549
1,AGO,2015,0.169137
2,AGO,2016,0.173008
3,AGO,2017,0.196649
4,AGO,2018,0.215053


In [849]:
SSA_country_DE_not_issued = SSA_country_DE_yearly[['ISO3code','year','Not_issued_rate']]
SSA_country_DE_not_issued.columns=(['ISO3code','year','Not_issued_rate_GER'])
SSA_country_DE_not_issued.head()

,ISO3code,year,Not_issued_rate_GER
0,AGO,2014,0.291944
1,AGO,2015,0.359645
2,AGO,2016,0.353857
3,AGO,2017,0.409730
4,AGO,2018,0.396119


In [851]:
SSA_country_not_issued = SSA_country_Schengen_not_issued.merge(SSA_country_DE_not_issued, on=['ISO3code','year'], how='left')

In [852]:
SSA_country_not_issued_2024 = SSA_country_not_issued[SSA_country_not_issued['year']=='2024']
SSA_country_not_issued_2024.head()

,ISO3code,year,Not_issued_rate_Schengen,Not_issued_rate_GER
10,AGO,2024,0.277918,0.441912
21,BDI,2024,0.391700,NaN
32,BEN,2024,0.249472,0.264596
43,BFA,2024,0.216854,0.194250
54,BWA,2024,0.161666,0.161666


In [853]:
upload_to_google_sheets(SPREADSHEET_URL,'SSA-rejection-rates-2024',SSA_country_not_issued_2024)